In [5]:
# %cd C:\Users\piotr\Desktop\PSG data
# %pip install -r requirements.txt

C:\Users\piotr\Desktop\PSG data


In [7]:
# import all the relevant libraries
import wfdb
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, hamming_loss
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import FunctionTransformer,StandardScaler
from mne.datasets import sample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from mne_features.feature_extraction import extract_features, FeatureExtractor
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest,f_classif
import neurokit2 as nk
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

ERROR: You must give at least one requirement to install (see "pip help install")


Note: you may need to restart the kernel to use updated packages.


In [100]:
def data_creation(record_path):
    # Load WFDB annotation
    annotation = wfdb.rdann(record_path, extension='apn')
    
    # Get the sampling frequency from the record
    sampling_rate = wfdb.rdrecord(record_path).fs  
    
    # Convert annotation sample indices to times in seconds
    annotation_times = np.array(annotation.sample) / sampling_rate  
    
    # Extract annotation labels and relable 0 for no apnea and 1 for apnea
    annotation_labels = np.array(annotation.symbol)  
    for i in range(len(annotation_labels)):
        if annotation_labels[i]=="N" :
            annotation_labels[i]=0
        if annotation_labels[i]=="A" :
            annotation_labels[i]=1
    annotation_labels=annotation_labels.astype(np.int32)
    # get the signal
    signal,dic = wfdb.rdsamp(record_path)
    signal=signal.flatten()
    
    #process the signal with nk
    ecg_signals, ecg_info=nk.ecg_process(signal,sampling_rate=sampling_rate)

    #create epochs and extract features:
    epoch_duration = 120  # seconds
    epoch_shift = 30  # seconds
    
    event_onsets = np.clip(annotation_times-epoch_shift,a_min=0,a_max=None)
    event_samples = (event_onsets * sampling_rate).astype(np.int32)
    
    # Use `nk.epochs_create()` to generate overlapping epochs
    epochs = nk.epochs_create(
        data=ecg_signals,
        events=event_onsets,  # Pass event locations
        sampling_rate=sampling_rate,
        epochs_start=0,  # Start at event onset
        epochs_end=epoch_duration,  # End 180s later
    )  
    df=nk.ecg_intervalrelated(epochs)
    df= df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)
    df = df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)
    df.dropna(axis=1,how="any",inplace=True)
    features= df.iloc[:, 1:].to_numpy()
    labels=annotation_labels
    return features,labels

In [ ]:
list_of_features=[]
list_of_labels=[]
for i in list_of_file_names:
    record_path=fr"C:\Users\piotr\Desktop\PSG data\Apnea ECG\{i}"
    features,labels = data_creation(record_path)
    list_of_features.append(features)
    list_of_labels.append(labels)

C:\Users\piotr\AppData\Local\Temp\ipykernel_16192\3390396646.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df= df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)
C:\Users\piotr\AppData\Local\Temp\ipykernel_16192\3390396646.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)
C:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:\Users\piotr\AppData\Local\Temp\ipykernel_16192\3390396646.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df= df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 

In [102]:
for i in list_of_file_names[0:1]:
    record_path=fr"C:\Users\piotr\Desktop\PSG data\Apnea ECG\{i}"
    features,labels = data_creation(record_path)

C:\Users\piotr\AppData\Local\Temp\ipykernel_16192\3390396646.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df= df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)
C:\Users\piotr\AppData\Local\Temp\ipykernel_16192\3390396646.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) == 1 else x)


In [103]:
featuares

NameError: name 'featuares' is not defined

In [105]:
features.shape

(489, 77)

In [106]:
labels.shape

(489,)